In [15]:
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt

def cut_criterion_interval(g_min, g_max, minMax, intervals):
    g = []
    for j in range(intervals + 1):
        g_i = g_min + ((j) / intervals) * (g_max - g_min)
        if minMax == "min":
            g.insert(0, g_i)
        elif minMax == "max":
            g.append(g_i)
    return g

def UTASTAR(Fu, Fu_ref, ranks, minMax, intervals, delta, acc):
    if Fu.shape[1] != Fu_ref.shape[1]:
        raise ValueError("The number of criteria in Fu and Fu_ref is different")

    # Dimensions
    il_pkt, il_kryt = Fu_ref.shape

    w = []
    for i in range(il_kryt):
        w_i = np.zeros((il_pkt, intervals[i]))
        g_i = cut_criterion_interval(min(Fu_ref[:, i]), max(Fu_ref[:, i]), minMax[i], intervals[i])
        
        for k in range(il_pkt):
            if Fu_ref[k, i] not in g_i:
                lower_b, upper_b = 0, 0
                for j in range(len(g_i)):
                    if Fu_ref[k, i] > g_i[j]:
                        if minMax[i] == "min":
                            lower_b, upper_b = j, j - 1
                        elif minMax[i] == "max":
                            lower_b, upper_b = j, j + 1
                
                for j in range(intervals[i] - 1):
                    w_i[k, j] = 1
                w_i[k, intervals[i] - 1] = (Fu_ref[k, i] - g_i[upper_b]) / (g_i[lower_b] - g_i[upper_b])
            else:
                index = g_i.index(Fu_ref[k, i])
                if index != 0:
                    for j in range(index):
                        w_i[k, j] = 1

        w.append(w_i)
    
    w = np.hstack(w)

    A, b, Aeq, beq = [], [], [], []
    for i in range(il_pkt - 1):
        if ranks[i] != ranks[i + 1]:
            A_i = w[i] - w[i + 1]
            A.append(A_i)
            b.append(delta)
        else:
            Aeq_i = w[i] - w[i + 1]
            Aeq.append(Aeq_i)
            beq.append(0)

    Aeq.append(np.ones(w.shape[1]))
    beq.append(1)
    A, b = -np.array(A), -np.array(b)
    Aeq, beq = np.array(Aeq), np.array(beq)
    lb = np.zeros(w.shape[1])
    ub = np.full(w.shape[1], np.inf)

    x = []
    tmp_start = 0
    for i in range(il_kryt):
        z = np.zeros(w.shape[1])
        z[tmp_start:tmp_start + intervals[i]] = -1
        tmp_start += intervals[i]

        # Correct bounds
        bounds = [(lb[j], ub[j]) for j in range(len(lb))]

        result = linprog(z, A_ub=A, b_ub=b, A_eq=Aeq, b_eq=beq, bounds=bounds, method="highs")
        
        if result.success:
            x.append(result.x)
        else:
            raise ValueError(f"Linear programming failed: {result.message}")


    x_avg = np.mean(x, axis=0)

    u_g = []
    tmp = 0
    for i in range(il_kryt):
        # plt.figure()
        
        g_i = cut_criterion_interval(min(Fu_ref[:, i]), max(Fu_ref[:, i]), minMax[i], intervals[i])
        u_g_i_j = [0]
        for j in range(1, len(g_i)):
            u_g_i_j.append(u_g_i_j[j - 1] + x_avg[tmp])
            tmp += 1

        x1 = np.linspace(g_i[0], g_i[-1], acc)
        y1 = np.interp(x1, g_i, u_g_i_j)
        # plt.plot(g_i, u_g_i_j, '*', label='Points')
        # plt.plot(x1, y1, label='Interpolated')
        # plt.xlim(min(g_i), max(g_i))
        # plt.ylim(0, 1)
        # plt.title(f"Partial utility function for criterion {i + 1}")
        # plt.legend()
        # plt.show()
        u_g.append((x1, y1))

    U = []
    for i in range(Fu.shape[0]):
        temp = 0
        for j in range(il_kryt):
            x_vals, y_vals = u_g[j]
            temp += np.interp(Fu[i, j], x_vals, y_vals)
        U.append(temp)

    ranking = np.argsort(-np.array(U)) + 1
    return U, u_g, ranking

In [16]:
Fu_ref = np.array([
    [3, 10, 1],
    [4, 20, 2],
    [2, 20, 0],
    [6, 40, 0],
    [30, 30, 3]
])

Fu = np.array([
    [3, 10, 1],
    [4, 20, 2],
    [2, 20, 0],
    [6, 40, 0],
    [30, 30, 3],
    [2, 10, 3],
    [5, 10, 0],
    [15, 20, 3],
    [13, 20, 0],
    [10, 35, 2],
    [30, 40, 0]
])

minMax = ["min", "min", "max"]
intervals = [2, 3, 3]
ranks = [1, 2, 2, 4, 5]
delta = 0.05
acc = 1000

# Run the algorithm
U, u_g, ranking = UTASTAR(Fu, Fu_ref, ranks, minMax, intervals, delta, acc)
print("U:", U)
print("Ranking:", ranking)

U: [0.35833333333333334, 0.016666666666666673, 0.50625, 0.0, 0.15208333333333332, 1.0, 0.3416666666666667, 0.15208333333333332, 0.0, 0.016666666666666673, 0.0]
Ranking: [ 6  3  1  7  5  8  2 10  4  9 11]
